In [1]:
from pathlib import Path
import pandas as pd
from typing import Optional, Dict
from functools import reduce
import geopandas as gpd

### Empirirca regio data

In [2]:
BASE_DIR = Path("..") / "data" / "input" / "empirica_regio_data"

def load_empirica_long(filename: str, region: Optional[str] = None, sheet_name: str = "Daten", value_col: str = "value") -> pd.DataFrame:
    
    path = BASE_DIR / filename

    df = pd.read_excel(path, sheet_name=sheet_name)

    year_cols = [
        c for c in df.columns
        if isinstance(c, (int, float)) or str(c).isdigit()
    ]

    df_long = df.melt(
        id_vars=["Regionsebene", "RegionID", "Regionsname"],
        value_vars=year_cols,
        var_name="Jahr",
        value_name=value_col
    )

    df_long["Jahr"] = df_long["Jahr"].astype(int)
    df_long[value_col] = pd.to_numeric(df_long[value_col], errors="coerce")
    df_long = df_long.sort_values(["RegionID", "Jahr"]).reset_index(drop=True)

    if region is not None:
        df_long = df_long[df_long["Regionsname"] == region].copy()

    return df_long

In [3]:
def load_empirica_multi(files: Dict[str, str], region: Optional[str] = None, sheet_name: str = "Daten") -> pd.DataFrame:

    dfs = []

    for value_col, filename in files.items():
        df = load_empirica_long(
            filename=filename,
            region=region,
            sheet_name=sheet_name,
            value_col=value_col
        )
        dfs.append(df)

    key_cols = ["Regionsebene", "RegionID", "Regionsname", "Jahr"]

    df_merged = reduce(
        lambda left, right: pd.merge(left, right, on=key_cols, how="outer"),
        dfs,
    )
    
    df_merged = df_merged.sort_values(["RegionID", "Jahr"]).reset_index(drop=True)
    
    return df_merged

In [4]:
files = {
    # Kaufpreise (m²-Preise ETW / EZFH, Perzentile)
    "Kaufpreis/m2 ETW 5%":       r"Kaufpreise\Anfangspreise_ETW_Insgesamt.xlsx",
    "Kaufpreis/m2 ETW 50%":      r"Kaufpreise\Standardpreise_ETW_Insgesamt.xlsx",
    "Kaufpreis/m2 ETW 95%":      r"Kaufpreise\Spitzenpreise_ETW_Insgesamt.xlsx",
    "Kaufpreis/m2 EZFH 5%":      r"Kaufpreise\Anfangspreise_EZFH_Insgesamt.xlsx",
    "Kaufpreis/m2 EZFH 50%":     r"Kaufpreise\Standardpreise_EZFH_Insgesamt.xlsx",
    "Kaufpreis/m2 EZFH 95%":     r"Kaufpreise\Spitzenpreise_EZFH_Insgesamt.xlsx",

    # Mieten (m²-Mieten, Perzentile)
    "Mietpreis/m2 5%":               r"Mietpreise\Anfangsmieten_Insgesamt.xlsx",
    "Mietpreis/m2 50%":              r"Mietpreise\Standardmieten_Insgesamt.xlsx",
    "Mietpreis/m2 95%":              r"Mietpreise\Spitzenmieten_Insgesamt.xlsx",

    # Rendite / Finanzierung / Belastung
    "Vervielfältiger":           r"Vervielfältiger.xlsx",
    "Bruttomietrendite":         r"Bruttomietrendite.xlsx",
    "Annuitätenbelastung ETW":   r"Annuitätenbelastung_ETW.xlsx",
    "Annuitätenbelastung EZFH":  r"Annuitätenbelastung_EZFH.xlsx",
    "Mietbelastung":             r"Mietbelastung.xlsx",

    # Demografie / Arbeitsmarkt
    "Einwohner":                 r"Einwohner.xlsx",
    "Durchschnittsalter":        r"Durchschnittsalter.xlsx",
    "Arbeitsvolumen Einwohner":  r"Arbeitsvolumen_Einwohner.xlsx",
    "Arbeitsvolumen Erwerbstätige": r"Arbeitsvolumen_Erwerbstätige.xlsx",
    "Arbeitsvolumen Haushalt":   r"Arbeitsvolumen_Haushalt.xlsx",

    # Einkommen / BIP
    "BIP insgesamt":             r"BIP_Insgesamt.xlsx",
    "Jahreseinkommen Einwohner": r"Jahreseinkommen_Einwohner.xlsx",
    "Jahreseinkommen Haushalt":  r"Jahreseinkommen_Haushalt.xlsx",

    # Wohnen / Wohnflächen / Wohnbestände
    "Wohneigentumsquote":        r"Wohneigentumsquote.xlsx",
    "Genehmigte Wohnungen":      r"Genehmigte_Wohnungen.xlsx",
    "Vermietete Wohnungen":      r"Vermietete_Wohnungen.xlsx",
    "Wohnfläche":                r"Wohnfläche.xlsx",
    "Wohnfläche insgesamt":      r"Wohnfläche_insgesamt.xlsx",
    "Wohnungen":                 r"Wohnungen.xlsx",
}

df_empirica_regio = load_empirica_multi(files)

### Macroeconomic data

In [5]:
BASE_DIR = Path("..") / "data" / "input" / "macroeconomic_data"

# Effektiver Jahreszins
df_immobilienzins = pd.read_csv(BASE_DIR / "Effektiver_Jahreszins.csv", header=None, names=["raw"], encoding="utf-8")
df_immobilienzins = df_immobilienzins["raw"].str.split(";", expand=True)
df_immobilienzins.columns = ["Datum", "Jahr", "Monat", "Zins"]
if df_immobilienzins.loc[0, "Datum"].lower().startswith("datum"):
    df_immobilienzins = df_immobilienzins.iloc[1:].reset_index(drop=True)
df_immobilienzins["Datum"] = pd.to_datetime(df_immobilienzins["Datum"])
df_immobilienzins["Jahr"] = df_immobilienzins["Jahr"].astype(int)
df_immobilienzins["Monat"] = df_immobilienzins["Monat"].astype(int)
df_immobilienzins["Zins"] = (df_immobilienzins["Zins"].astype(str).str.replace(",", ".", regex=False).astype(float))
df_immobilienzins_jährlich = (df_immobilienzins.groupby("Jahr")["Zins"].mean().round(2).reset_index())

# S&P500 data

### Geo data

In [6]:
BASE_GEO = Path("..") / "data" / "input" / "geo_data"
SHAPE_GEM = BASE_GEO / "vg250_01-01.tm32.shape.ebenen" / "vg250_ebenen_0101" / "VG250_GEM.shp"

gdf_gem = gpd.read_file(SHAPE_GEM)
gdf_gem = gdf_gem.to_crs(epsg=4326)
gdf_gem["lon"] = gdf_gem.geometry.centroid.x
gdf_gem["lat"] = gdf_gem.geometry.centroid.y
df_gemeinden_coords = gdf_gem[["AGS", "lat", "lon"]].copy()
df_gemeinden_coords = df_gemeinden_coords.rename(columns={"AGS": "RegionID"})
df_gemeinden_coords["RegionID"] = df_gemeinden_coords["RegionID"].astype("int64")

C:\Users\49176\AppData\Local\Temp\ipykernel_31128\3649422146.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_gem["lon"] = gdf_gem.geometry.centroid.x
C:\Users\49176\AppData\Local\Temp\ipykernel_31128\3649422146.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_gem["lat"] = gdf_gem.geometry.centroid.y


### Save csv

In [7]:
OUTPUT_DIR = Path("..") / "data" / "output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

df_empirica_regio.to_csv(OUTPUT_DIR / "empirica_regio_data.csv", index=False, encoding="utf-8-sig")

df_gemeinden_coords.to_csv(OUTPUT_DIR / "Gemeinden_coordinates.csv", index=False, encoding="utf-8-sig")

df_immobilienzins_jährlich.to_csv(OUTPUT_DIR / "zinsdaten.csv", index=False, encoding="utf-8-sig")
